In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv('Scrapped_Data/Merge_data_cybSec_ancienne/Apec_data_total_merged_filred_cleaned.csv')
df

,entreprise,type_contrat,date_publication,date_mise_a_jour,salaire,prise_de_poste,experience,metier,statut_du_poste,zone_de_deplacement,...,profil_recherche,langues,savoir_etre,savoir_faire,descriptif_entreprise,nombre_posts,duree,ville,code_postal,departement
0,AI&Data,CDI,2024-02-01,2024-02-01,NaN,Dès que possible,2,Data analyst,Cadre du secteur privé,Nationale,...,De formation supérieure en statistiques et mat...,['Aucune langue attendue'],"['Capacité à prendre des décisions', ""Capacité...","['Algorithme', 'Amazon Web Services', 'Base de...",AI&Data est une Agence Data sur trois domaines...,1,NaN,Boulogne-Billancourt,92,Hauts-de-Seine
1,BLUE CONSULTING,CDI,2023-11-29,2024-01-17,55.0,02/01/2024,5,Data analyst,Cadre du secteur privé,Régionale,...,Savoir-être :\n- Avoir le sens de l écoute et ...,['Aucune langue attendue'],"[""Capacité d'analyse"", 'Capacité d’écoute', 'C...","['Analyse de données', 'Analyse technique', 'D...","La société BLUE CONSULTING, Entreprise de serv...",1,NaN,Levallois-Perret,92,Hauts-de-Seine
2,CAF Hauts-de-Seine,CDI,2024-02-07,2024-02-07,34.0,Dès que possible,0,Data analyst,Cadre du secteur privé,Nationale,...,Un niveau bac+ 3 souhaité dans le domaine écon...,['Aucune langue attendue'],['Aucun savoir-être attendu'],"['Analyse de données', 'Analyse financière', '...","La Caf des Hauts-de-Seine, c’est...\n630 colla...",1,NaN,Nanterre,92,Hauts-de-Seine
3,Avanade,CDI,2024-01-16,2024-01-16,65.0,Dès que possible,7,Chief Data Officer,Cadre du secteur privé,Régionale,...,De façon concrète \n En tant que Manager Data ...,"['Français', 'Anglais']","[""Esprit d'équipe"", 'Leadership']","['Communication', 'Gestion de projet', 'Règlem...","Avanade France, des experts Microsoft et ses p...",1,NaN,Issy-les-Moulineaux,92,Hauts-de-Seine
4,Lydia CHERFI,CDI,2024-01-24,2024-01-24,NaN,Dès que possible,4,Data analyst,Cadre du secteur privé,Nationale,...,Vous avez acquis au moins 4 ans des compétence...,['Aucune langue attendue'],['Aucun savoir-être attendu'],['Aucun savoir-faire attendu'],"MAKE IT HAPPEN - Depuis 2001, PREREQUIS mise s...",1,NaN,Neuilly-sur-Seine,92,Hauts-de-Seine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,MICHAEL PAGE INTERNATIONAL FRANCE,CDI,2024-03-16,2024-03-16,52.5,Dès que possible,0,Responsable sécurité informatique,Cadre du secteur privé,Pas de déplacement,...,Ingénieur ou issu d'une formation supérieure (...,['Aucune langue attendue'],"[""Esprit d'équipe"", 'Rigueur', ""Sens de l'orga...","['Administration système et réseau', 'Analyse ...",Le SDIS 42 (Service Départemental d'Incendie e...,1,NaN,Saint-Étienne,42,Loire
3680,Siléane Saint-Etienne,CDI,2024-03-05,2024-03-07,NaN,Dès que possible,5,Développeur,Cadre du secteur privé,Union Européenne,...,Votre profil\n Vous avez obtenu un diplôme en...,['Aucune langue attendue'],"['Autonomie', 'Capacité de communication', 'Fo...","['C sharp', 'C++', 'Management des personnes',...","SILEANE spécialiste en ROBOTIQUE et VISION, en...",1,NaN,Saint-Étienne,42,Loire
3681,AKKODIS CLERMONT-FERRAND,CDI,2024-03-07,2024-03-07,NaN,Dès que possible,3,Ingénieur système,Cadre du secteur privé,Pas de déplacement,...,Votre profil:\nDe formation supérieure en écol...,['Anglais'],"['Agilité', 'Ambition', 'Audace', 'Créativité'...","['Réseau électrique', 'Accompagnement du clien...","Akkodis, est un acteur mondial de l'ingénierie...",1,NaN,Roanne,42,Loire
3682,EURENCO,CDI,2024-02-19,2024-02-19,46.0,Dès que possible,0,Architecte infrastructures,Cadre du secteur privé,Pas de déplacement,...,De formation supérieure en Informatique (BAC+3...,['Aucune langue attendue'],"['Autonomie', 'Dynamisme', ""Esprit d'équipe"", ...","['Qualité', 'Archivage', 'Certification ITIL',...",Leader européen dans le domaine des explosifs ...,1,NaN,Bergerac,24,Dordogne


In [6]:
moyenne_salaire_par_departement = df.groupby("departement")["salaire"].mean().reset_index()
moyenne_salaire_par_departement.isna().sum()

departement     0
salaire        10
dtype: int64

### Remplacer les valeurs NaN par la moyenne générale des salaires

In [7]:
moyenne_generale_salaire = df["salaire"].mean()
moyenne_salaire_par_departement["salaire"].fillna(moyenne_generale_salaire, inplace=True)
moyenne_salaire_par_departement


,departement,salaire
0,Ain,48.700000
1,Allier,47.728255
2,Alpes-Maritimes,43.476562
3,Ardèche,42.666667
4,Ariège,47.728255
...,...,...
81,Vendée,45.772727
82,Vienne,35.000000
83,Vosges,41.250000
84,Yonne,47.728255


In [8]:
for dep in moyenne_salaire_par_departement["departement"].unique() :
    mask = df["departement"] == dep
    new_salaire = moyenne_salaire_par_departement.loc[moyenne_salaire_par_departement["departement"] == dep, "salaire"].iloc[0]
    df.loc[mask, "salaire"] = df.loc[mask, "salaire"].fillna(new_salaire)

df.isna().sum()

entreprise                  0
type_contrat                0
date_publication            0
date_mise_a_jour            0
salaire                     0
prise_de_poste              0
experience                  0
metier                      0
statut_du_poste             0
zone_de_deplacement         0
secteur_d_activite          0
Teletravail              1919
descriptif_poste            0
profil_recherche            0
langues                     0
savoir_etre                 0
savoir_faire                0
descriptif_entreprise       0
nombre_posts                0
duree                    3462
ville                       0
code_postal                 0
departement                 0
dtype: int64

In [9]:
features = df[['entreprise', 'type_contrat', 'experience', 'langues', 'savoir_etre', 'savoir_faire']]

# Traiter les données manquantes si nécessaire
features.fillna('', inplace=True)

# Traitement des données textuelles
vectorizer = TfidfVectorizer(stop_words='french')
X = vectorizer.fit_transform(features.apply(lambda x: ' '.join(x), axis=1))

# Normalisation des données
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X.toarray())

# Appliquer le clustering (par exemple, avec K-means)
kmeans = KMeans(n_clusters=10, random_state=42)
clusters = kmeans.fit_predict(X_normalized)

# Ajouter les informations de cluster au DataFrame
df['cluster'] = clusters

/tmp/ipykernel_88654/1276776810.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna('', inplace=True)


TypeError: sequence item 2: expected str instance, int found